# Recommendation System for Online Market Fresh Seafood: Manettas
## with Content Based Filtering (CBF)

In [54]:
import   random
import   pickle
import   warnings
warnings.filterwarnings(action='ignore')

In [2]:
from pyforest                        import *
from sklearn.metrics                 import precision_score, recall_score, f1_score
from nltk.stem.porter                import PorterStemmer
from sklearn.metrics.pairwise        import cosine_similarity, linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer


In [3]:
df   = pd.read_csv('C:/Users/ASUS/kode/Projek/Recommendation_System_Manettas/manettas.csv')
df

<IPython.core.display.Javascript object>

,nama_produk,harga,kriteria,asal_produk,link,deskripsi,kategori
0,Salmon Fillets (2 x 180g),$23.90 per pack,Each pack contains 2 x 180g fillets,"Origin: Tasmania, Australia",https://www.manettas.com.au/product/salmon-fil...,Catch salmon fillet in Tasmania's cold waters....,"Fish - Fillets, Cutlets & Tails"
1,Bulk Buy Salmon Portions,$59.90 per kg,Minimum Order for Bulk Buy is 5kg,Origin: Australia,https://www.manettas.com.au/product/bulk-buy-s...,Wholesale Salmon Portions are perfect for eith...,"Fish - Fillets, Cutlets & Tails"
2,Tasmanian Salmon Side,$78.00 per side,Minimum weight per side 1.4kg,"Origin: Tasmania, Australia",https://www.manettas.com.au/product/tasmanian-...,Tasmanian Salmon Side is a diverse fish in the...,"Fish - Fillets, Cutlets & Tails"
3,Bulk Buy Salmon Side,$49.90 per kg,Minimum Order for Bulk Buy is 10kg,Origin: Australia,https://www.manettas.com.au/product/bulk-buy-s...,Wholesale salmon side through bulk buy is perf...,"Fish - Fillets, Cutlets & Tails"
4,Ora King Salmon Side,$110.00 per side,Minimum weight per side 1.5kg,Origin: New Zealand,https://www.manettas.com.au/product/ora-king-s...,This King Salmon from New Zealand is regarded ...,"Fish - Fillets, Cutlets & Tails"
...,...,...,...,...,...,...,...
434,Hiramasa Kingfish Fillets,$10.50 per fillet (25% SAVINGS),Minimum weight per fillet 200g,Origin: Australia,https://www.manettas.com.au/product/kingfish-f...,By far the premium name in Kingfish is Hiramas...,Manettas Weekly Specials
435,Hiramasa Kingfish Side Frozen,$45.00 per side (18% SAVINGS),Minimum weight per side 1kg,Origin: Australia,https://www.manettas.com.au/product/hiramasa-k...,"Also known as yellowtail kingfish, this is a v...",Manettas Weekly Specials
436,Portoro Scotch Fillets MB4+ (2 x 300g),$45.00 per pack (23% SAVINGS),Scotch fillets are packed 2 x 300g,Origin: Australia,https://www.manettas.com.au/product/portoro-sc...,Indulge in this exquisite Portoro Scotch Fille...,Manettas Weekly Specials
437,T-Bone (2 x 400g),$35.00 per pack (26% SAVINGS),T-Bones are packed 2 x 400g,Origin: Australia,https://www.manettas.com.au/product/t-bone-2-x...,T-Bone (2 x 400g) is a premium cut of meat wit...,Manettas Weekly Specials


In [4]:
df['deskripsi'][0]

"Catch salmon fillet in Tasmania's cold waters. These are the finest Australian Salmon Fillets in the market. All Tasmanian salmon is completely sustainable. Salmon (in particular Tasmanian Salmon) is one of the healthiest meals you can prepare.\r\n\r\nFresh seafood online that delivers direct from the Sydney Fish Market. Buy salmon fillet online today, learn how to cook salmon fillets, work out how to achieve that crispy skin salmon, and looks at the best sauce for salmon fillet. This is the fresh seafood online that delivers.How Do My Tasmanian Salmon Fillet come:each pack contains 2 x 180g filletsall fresh fillets are skin onThe minimum weight of 360gm at $68.00/kg in used to calculate the fixed priceWant to learn how to cook\xa0salmon fillets, how to get the crispy skin salmon or even best sauce for salmon fillet.Click here to learn how to cook fish fillets.View our Crispy Skin Salmon recipe here."

### Data Preprocessing
- Cleaning Data missing value
- Seleksi Fitur: Memilih fitur yang relevan dan berguna untuk proses pemodelan.
- Pembersihan Kolom 'asal_produk': Menghapus kata 'Origin: ' dari kolom 'asal_produk' dan mengonversi/merubahnya menjadi list [,] menggunakan metode split.
- Pembersihan Deskripsi: Mengonversi/merubah kolom 'deskripsi' menjadi list menggunakan metode split.
- Penggabungan Fitur: Menggabungkan semua fitur yang digunakan ke dalam satu fitur utama yang mengandung informasi selain 'nama_produk'.
- Stemming: Melakukan proses stemming untuk menormalkan kata-kata dalam dataset.
- Lowercasing: Mengubah semua teks menjadi huruf kecil untuk konsistensi.
- Count Vectorizer: Mengubah teks menjadi representasi numerik menggunakan Count Vectorizer untuk persiapan pemodelan.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439 entries, 0 to 438
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nama_produk  439 non-null    object
 1   harga        439 non-null    object
 2   kriteria     399 non-null    object
 3   asal_produk  428 non-null    object
 4   link         439 non-null    object
 5   deskripsi    439 non-null    object
 6   kategori     439 non-null    object
dtypes: object(7)
memory usage: 24.1+ KB


In [6]:
df2 = df.copy()
df2 = df2[['nama_produk',
           'asal_produk',
           'deskripsi',
           'kategori']]

In [7]:
df2

,nama_produk,asal_produk,deskripsi,kategori
0,Salmon Fillets (2 x 180g),"Origin: Tasmania, Australia",Catch salmon fillet in Tasmania's cold waters....,"Fish - Fillets, Cutlets & Tails"
1,Bulk Buy Salmon Portions,Origin: Australia,Wholesale Salmon Portions are perfect for eith...,"Fish - Fillets, Cutlets & Tails"
2,Tasmanian Salmon Side,"Origin: Tasmania, Australia",Tasmanian Salmon Side is a diverse fish in the...,"Fish - Fillets, Cutlets & Tails"
3,Bulk Buy Salmon Side,Origin: Australia,Wholesale salmon side through bulk buy is perf...,"Fish - Fillets, Cutlets & Tails"
4,Ora King Salmon Side,Origin: New Zealand,This King Salmon from New Zealand is regarded ...,"Fish - Fillets, Cutlets & Tails"
...,...,...,...,...
434,Hiramasa Kingfish Fillets,Origin: Australia,By far the premium name in Kingfish is Hiramas...,Manettas Weekly Specials
435,Hiramasa Kingfish Side Frozen,Origin: Australia,"Also known as yellowtail kingfish, this is a v...",Manettas Weekly Specials
436,Portoro Scotch Fillets MB4+ (2 x 300g),Origin: Australia,Indulge in this exquisite Portoro Scotch Fille...,Manettas Weekly Specials
437,T-Bone (2 x 400g),Origin: Australia,T-Bone (2 x 400g) is a premium cut of meat wit...,Manettas Weekly Specials


In [8]:
df2.isnull().sum()

nama_produk     0
asal_produk    11
deskripsi       0
kategori        0
dtype: int64

In [9]:
df2.duplicated().sum()

0

In [10]:
df2 = df2.dropna()

In [11]:
df2['asal_produk'] = df2['asal_produk'].str.replace('Origin: ', '')

In [12]:
df2['asal_produk'] = df2['asal_produk'].str.replace('', '')

In [13]:
df2['asal_produk'] = df2['asal_produk'].apply(lambda x: x.split())

In [14]:
df2['deskripsi']   = df2['deskripsi'].apply(lambda x: x.split())

In [15]:
df2['kategori']    = df2['kategori'].apply(lambda x: x.split())

In [16]:
df2['tags']        = df2['asal_produk'] + df2['deskripsi'] + df2['kategori']

In [17]:
df2

,nama_produk,asal_produk,deskripsi,kategori,tags
0,Salmon Fillets (2 x 180g),"[Tasmania,, Australia]","[Catch, salmon, fillet, in, Tasmania's, cold, ...","[Fish, -, Fillets,, Cutlets, &, Tails]","[Tasmania,, Australia, Catch, salmon, fillet, ..."
1,Bulk Buy Salmon Portions,[Australia],"[Wholesale, Salmon, Portions, are, perfect, fo...","[Fish, -, Fillets,, Cutlets, &, Tails]","[Australia, Wholesale, Salmon, Portions, are, ..."
2,Tasmanian Salmon Side,"[Tasmania,, Australia]","[Tasmanian, Salmon, Side, is, a, diverse, fish...","[Fish, -, Fillets,, Cutlets, &, Tails]","[Tasmania,, Australia, Tasmanian, Salmon, Side..."
3,Bulk Buy Salmon Side,[Australia],"[Wholesale, salmon, side, through, bulk, buy, ...","[Fish, -, Fillets,, Cutlets, &, Tails]","[Australia, Wholesale, salmon, side, through, ..."
4,Ora King Salmon Side,"[New, Zealand]","[This, King, Salmon, from, New, Zealand, is, r...","[Fish, -, Fillets,, Cutlets, &, Tails]","[New, Zealand, This, King, Salmon, from, New, ..."
...,...,...,...,...,...
434,Hiramasa Kingfish Fillets,[Australia],"[By, far, the, premium, name, in, Kingfish, is...","[Manettas, Weekly, Specials]","[Australia, By, far, the, premium, name, in, K..."
435,Hiramasa Kingfish Side Frozen,[Australia],"[Also, known, as, yellowtail, kingfish,, this,...","[Manettas, Weekly, Specials]","[Australia, Also, known, as, yellowtail, kingf..."
436,Portoro Scotch Fillets MB4+ (2 x 300g),[Australia],"[Indulge, in, this, exquisite, Portoro, Scotch...","[Manettas, Weekly, Specials]","[Australia, Indulge, in, this, exquisite, Port..."
437,T-Bone (2 x 400g),[Australia],"[T-Bone, (2, x, 400g), is, a, premium, cut, of...","[Manettas, Weekly, Specials]","[Australia, T-Bone, (2, x, 400g), is, a, premi..."


In [18]:
df_baru = df2[['nama_produk', 'tags']]

In [19]:
df_baru

,nama_produk,tags
0,Salmon Fillets (2 x 180g),"[Tasmania,, Australia, Catch, salmon, fillet, ..."
1,Bulk Buy Salmon Portions,"[Australia, Wholesale, Salmon, Portions, are, ..."
2,Tasmanian Salmon Side,"[Tasmania,, Australia, Tasmanian, Salmon, Side..."
3,Bulk Buy Salmon Side,"[Australia, Wholesale, salmon, side, through, ..."
4,Ora King Salmon Side,"[New, Zealand, This, King, Salmon, from, New, ..."
...,...,...
434,Hiramasa Kingfish Fillets,"[Australia, By, far, the, premium, name, in, K..."
435,Hiramasa Kingfish Side Frozen,"[Australia, Also, known, as, yellowtail, kingf..."
436,Portoro Scotch Fillets MB4+ (2 x 300g),"[Australia, Indulge, in, this, exquisite, Port..."
437,T-Bone (2 x 400g),"[Australia, T-Bone, (2, x, 400g), is, a, premi..."


In [20]:
df_baru['tags'] = df_baru['tags'].apply(lambda x: " ".join(x))

In [21]:
df_baru

,nama_produk,tags
0,Salmon Fillets (2 x 180g),"Tasmania, Australia Catch salmon fillet in Tas..."
1,Bulk Buy Salmon Portions,Australia Wholesale Salmon Portions are perfec...
2,Tasmanian Salmon Side,"Tasmania, Australia Tasmanian Salmon Side is a..."
3,Bulk Buy Salmon Side,Australia Wholesale salmon side through bulk b...
4,Ora King Salmon Side,New Zealand This King Salmon from New Zealand ...
...,...,...
434,Hiramasa Kingfish Fillets,Australia By far the premium name in Kingfish ...
435,Hiramasa Kingfish Side Frozen,"Australia Also known as yellowtail kingfish, t..."
436,Portoro Scotch Fillets MB4+ (2 x 300g),Australia Indulge in this exquisite Portoro Sc...
437,T-Bone (2 x 400g),Australia T-Bone (2 x 400g) is a premium cut o...


### Stemming
Stemming adalah proses menghilangkan afiks (awalan, akhiran, dan infiks) dari kata untuk menghasilkan bentuk dasarnya, yang disebut "kata dasar" atau "kata dasar". Tujuannya adalah untuk mengonversi kata-kata yang berbeda tetapi memiliki akar yang sama menjadi bentuk dasar yang seragam. Contohnya, kata-kata "berlari", "berlari", dan "berlari" semuanya akan diubah menjadi "lari" setelah proses stemming.

In [22]:
ps = PorterStemmer()

In [23]:
def stem(teks):
    y = []
    
    for i in teks.split():
        
        y.append(ps.stem(i))
        
    return " ".join(y)

In [24]:
df_baru['tags'] = df_baru['tags'].apply(stem)

In [25]:
df_baru['tags'][0]

"tasmania, australia catch salmon fillet in tasmania' cold waters. these are the finest australian salmon fillet in the market. all tasmanian salmon is complet sustainable. salmon (in particular tasmanian salmon) is one of the healthiest meal you can prepare. fresh seafood onlin that deliv direct from the sydney fish market. buy salmon fillet onlin today, learn how to cook salmon fillets, work out how to achiev that crispi skin salmon, and look at the best sauc for salmon fillet. thi is the fresh seafood onlin that delivers.how do my tasmanian salmon fillet come:each pack contain 2 x 180g filletsal fresh fillet are skin onth minimum weight of 360gm at $68.00/kg in use to calcul the fix pricew to learn how to cook salmon fillets, how to get the crispi skin salmon or even best sauc for salmon fillet.click here to learn how to cook fish fillets.view our crispi skin salmon recip here. fish - fillets, cutlet & tail"

In [26]:
# lowercase dulu untuk mastiin aja
df_baru['tags'] = df_baru['tags'].apply(lambda x: x.lower())

In [27]:
df_baru['tags'][1]

'australia wholesal salmon portion are perfect for either a larg get together, a restaurant, a hotel or for you as a caterer. order salmon with manetta seafood market and allow us to hand pick the fish from the best farm across the country. we are the freshest supplier of seafood from boat to your door. order fresh seafood onlin today and have it deliv to your door with manetta seafood market. deliveri in sydney is 7 day per week, so never go without your favourit fish. for wholesal seafood check out ourmanetta wholesal platformhow do my bulk buy wholesal salmon portion come:approxim 180g per portionskin on fish - fillets, cutlet & tail'

### Vector Representation dengan Count Vectorizer
mengubah teks menjadi representasi numerik yang dapat dimengerti oleh model mesin

In [28]:
cv     = CountVectorizer(max_features=5000, stop_words='english')

<IPython.core.display.Javascript object>

seperti biasa, mencoba 5000 fitur terlebih dahulu tergantun kapabilitas/

In [29]:
vektor = cv.fit_transform(df_baru['tags']).toarray()

In [30]:
vektor

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [31]:
vektor[0]

array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [32]:
cv.get_feature_names_out()

array(['00', '0kgpack', '10', ..., 'zest', 'zesti', 'zingy'], dtype=object)

### Kemiripan
cosine similarity itu untuk mengukur seberapa mirip dua vektor berada dalam ruang multidimensi array.

Setiap dimensi dalam vektor mewakili kata atau fitur tertentu, dan nilai dalam dimensi tersebut menunjukkan seberapa sering kata tersebut muncul dalam teks. Cosine Similarity mengukur sudut kosinus antara dua vektor. Semakin dekat sudut antara dua vektor, semakin mirip (atau serupa) kedua vektor tersebut. Nilai Cosine Similarity berkisar antara -1 hingga 1, di mana 1 menunjukkan kesamaan sempurna, 0 menunjukkan ketidaksamaan, dan -1 menunjukkan ketidaksamaan sempurna.

In [33]:
cosine_similarity(vektor).shape

(428, 428)

In [34]:
kesamaan = cosine_similarity(vektor)

In [35]:
kesamaan[0]

array([1.        , 0.39948181, 0.53687872, 0.54191949, 0.47621415,
       0.55481344, 0.36873292, 0.36433556, 0.34559738, 0.34632227,
       0.28602759, 0.39029614, 0.37548375, 0.28995744, 0.32740805,
       0.31154892, 0.48133629, 0.18722266, 0.15446526, 0.13078895,
       0.21163431, 0.052484  , 0.3621458 , 0.26737175, 0.14165399,
       0.13005917, 0.22396889, 0.26126925, 0.26157459, 0.14517994,
       0.14657088, 0.13624274, 0.28759865, 0.16604516, 0.13005917,
       0.16883808, 0.11492618, 0.17192455, 0.16227602, 0.3007261 ,
       0.33283077, 0.18656751, 0.21071848, 0.23232992, 0.22344798,
       0.26348233, 0.08888907, 0.54495396, 0.44469666, 0.1760493 ,
       0.18398704, 0.13597095, 0.16378594, 0.15912354, 0.15353567,
       0.16788059, 0.17548413, 0.16662473, 0.15725244, 0.17903885,
       0.0996271 , 0.07965213, 0.11612843, 0.10874613, 0.18479353,
       0.16765945, 0.13275355, 0.1548003 , 0.18153254, 0.20411826,
       0.04210013, 0.09265132, 0.10649964, 0.12767576, 0.01174

- enumerate untuk menghasilkan urutan atau no indeks
- reverse = True itu dalam urutan yang tinggi ke terendah
- fungsi lambda itu untuk mengurutkan daftar, jadi ambil elemen kedua (x[1]) dari stiap pasangan sebagai kunci pengurutn kita sampai 5 terats.

In [36]:
# mencari top 5 menu berdasarkan suatu produk

sorted(list(enumerate(kesamaan[0])), reverse=True, key=lambda x: x[1])[1:6]

[(5, 0.554813435013022),
 (47, 0.5449539581795492),
 (3, 0.5419194910995321),
 (2, 0.5368787216615044),
 (189, 0.5349224043153104)]

Hasil kesamaan tersebut cukup baik dikarenakan pada rentang nilai 0-1, khususnya di sekitar 5,3 - 5,5. Mungkin aku bisa ningkatin jumlah top produk yang memiliki kesamaan. Namun berdasarkan try dan error, mungkin kita bisa menampilkan rekomendasi 5 atau top 6 saja.

In [37]:
sorted(list(enumerate(kesamaan[0])), reverse=True, key=lambda x: x[1])[1:7]

[(5, 0.554813435013022),
 (47, 0.5449539581795492),
 (3, 0.5419194910995321),
 (2, 0.5368787216615044),
 (189, 0.5349224043153104),
 (16, 0.4813362872636812)]

In [38]:
def rekomendasi_produk(produk_seafood):
    produk_indeks      = df_baru[df_baru['nama_produk'] == produk_seafood].index[0]
    jarak              = kesamaan[produk_indeks] # sebuah list
    daftar_produk      = sorted(list(enumerate(jarak)), reverse=True, key=lambda x: x[1])[1:7]
    rekomendasi_df     = pd.DataFrame(columns=['Nama Produk', 'Nilai Kesamaan'])
    
    for i in daftar_produk:
        nama_produk    = df_baru.iloc[i[0]].nama_produk
        nilai_kesamaan = i[1]
        rekomendasi_df = pd.concat([rekomendasi_df, pd.DataFrame({'Nama Produk': [nama_produk],
                                                                  'Nilai Kesamaan': [nilai_kesamaan]})],
                                                                   ignore_index=True)
    
    return rekomendasi_df

In [39]:
df['asal_produk'][152]

'Origin: Australia'

In [40]:
df['nama_produk'][152]

'Divine Seafood Platter'

In [41]:
df['harga'][152]

'$199.95 per platter'

In [42]:
rekomendasi_produk('Divine Seafood Platter')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Nama Produk,Nilai Kesamaan
0,Seafood Skewers,0.981052
1,Seafood Skewers,0.977639
2,Seafood Skewers,0.977301
3,Seafood Skewers,0.969925
4,Fresh Marinara Mix,0.744954
5,Fresh Marinara Mix,0.736036


In [43]:
df['asal_produk'][152]

'Origin: Australia'

#### Perbandingan
Jika dibandingkan pada website tersebut ketika mengunjungi produk 'Divine Seafood Platter' itu hanya menampilkan dua rekomendasi produk yaitu 'Half Shell New Zealand Mussles' dan 'Ora King Salmon Caviar'. 

Jika dilakukan analisis untuk membuktikan apakah website tersebut memiliki sistem rekomenadasi dari beberapa hal dibawah ini:
- asumsikan bahwa website tersebut menggunakan fitur 'harga' untuk rekomendasi, namun harga produk dan satu rekomendasi produk di website tersebut terbilang jauh berbeda.
- dilihat dari tempat juga terbilang berbeda, karena produk itu dari Australia, namun rekomendasi itu dari New Zealand

Artinya sistem rekomendasi dari website tersebut belum terbilang optimal.


Maka dari itu dengan menerapkan sistem rekomendasi dengan content based filtering ini akan cukup membantu dalam menyarankan produk yang serupa terhadap suatu produk yang dipilih.

In [44]:
df['nama_produk'][431]

'Black Cod Fillet'

In [45]:
rekomendasi_produk('Local Sprat (Whitebait)')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Nama Produk,Nilai Kesamaan
0,Western Australia Whitebait,0.982194
1,Local Sprat (Whitebait),0.774389
2,Local Sprat (Whitebait),0.751916
3,Silver Fish,0.724329
4,Silver Fish,0.701766
5,Bulk Buy Leather Jackets,0.466041


In [46]:
rekomendasi_produk('Bulk Buy GIANT King Prawns Raw')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Nama Produk,Nilai Kesamaan
0,Giant Garlic Butter Prawns,0.991961
1,Garlic Marinated Prawns,0.617862
2,Garlic Marinated Prawns,0.613211
3,GIANT King Prawns Green,0.538482
4,Chilli Garlic Marinated Prawns,0.482640
5,King Prawn Green Medium-Large,0.478651


In [47]:
rekomendasi_produk('Black Cod Fillet')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Nama Produk,Nilai Kesamaan
0,Hiramasa Kingfish Fillets,0.970925
1,Hiramasa Kingfish Side Frozen,0.680490
2,Hiramasa Kingfish Side Frozen,0.634338
3,Flathead Fillets,0.461301
4,Barramundi Fillets (2 x 180g),0.429914
5,Kingfish Sashimi Loin,0.421070


In [65]:
def rekomendasi_acak(df, kesamaan, jumlah_produk):

    # jumlah produk yang dipilih
    nama_produk_acak                = random.sample(df['nama_produk'].tolist(),jumlah_produk)
    hasil_rekomendasi               = []

    for nama_produk in nama_produk_acak:
        
        # akses indeks
        produk_indeks               = df[df['nama_produk'] == nama_produk].index[0]

        # top 6 rekomendasi
        daftar_produk               = sorted(list(enumerate(kesamaan[produk_indeks])),
                                             reverse=True,
                                             key=lambda x: x[1])[1:7]
        
        rekomendasi_produk          = []

        # hasil rekomendasi tambahkan ke list
        for i, (indeks_produk, nilai_kesamaan) in enumerate(daftar_produk):
            
            nama_produk_rekomendasi = df.iloc[indeks_produk].nama_produk
            rekomendasi_produk.append((nama_produk_rekomendasi, nilai_kesamaan))

        # list hasil rekomendasi
        hasil_rekomendasi.append((nama_produk, rekomendasi_produk))

    # list untuk dataframe hasil rekomendasi
    list_hasil_rekomendasi          = []

    # average nilai kesamaan dari setiap tingkatan rekomendasi produk
    for i, (nama_produk, rekomendasi_produk) in enumerate(hasil_rekomendasi):
        for j, (nama_produk_rekomendasi, nilai_kesamaan) in enumerate(rekomendasi_produk):
            
            # syarat jika belum ada entri produk rekomendasi, bikin entri baru
            if len(list_hasil_rekomendasi) <= j:
                list_hasil_rekomendasi.append({'Tingkatan Produk': j + 1,
                                               'Rata-rata Kesamaan': nilai_kesamaan,
                                               'Jumlah Produk': 1})
            else:
                # perbaruin average dan jumlah produk yang dipilih secara acak
                list_hasil_rekomendasi[j]['Rata-rata Kesamaan'] += nilai_kesamaan
                list_hasil_rekomendasi[j]['Jumlah Produk'] += 1

    # tinggal dihitung rata-ratanya
    for produk in list_hasil_rekomendasi:
        produk['Rata-rata Kesamaan'] /= produk['Jumlah Produk']

    # dataframe
    df_rekomendasi = pd.DataFrame(list_hasil_rekomendasi)

    return df_rekomendasi

In [61]:
acak = rekomendasi_acak(df_baru, kesamaan)

<IPython.core.display.Javascript object>

In [62]:
acak

,Tingkatan Produk,Rata-rata Kesamaan,Jumlah Produk
0,1,0.870145,5
1,2,0.734443,5
2,3,0.587334,5
3,4,0.475855,5
4,5,0.461488,5
5,6,0.373071,5


In [63]:
acak2 = rekomendasi_acak(df_baru, kesamaan)

<IPython.core.display.Javascript object>

In [64]:
acak2

,Tingkatan Produk,Rata-rata Kesamaan,Jumlah Produk
0,1,0.812342,5
1,2,0.582792,5
2,3,0.506836,5
3,4,0.439675,5
4,5,0.425619,5
5,6,0.411330,5


In [66]:
acak30 = rekomendasi_acak(df_baru, kesamaan, 30)

<IPython.core.display.Javascript object>

In [67]:
acak30

,Tingkatan Produk,Rata-rata Kesamaan,Jumlah Produk
0,1,0.815634,30
1,2,0.671630,30
2,3,0.601856,30
3,4,0.483401,30
4,5,0.450481,30
5,6,0.436933,30


### Kesimpulan
Rata-rata nilai kesamaan dari setiap hasil rekomendasi terhadap 30 produk seafood yang dipilih secara acak pada website **Manettas** menunjukkan hasil yang sangat menarik. Dengan nilai rata-rata kesamaan atau cosine similarity pada produk rekomendasi terbaik itu sekitar 0.81, dapat disimpulkan bahwa produk rekomendasi terbaik cenderung memiliki kemiripan yang tinggi dengan produk yang dipilih. Hal ini mengindikasikan bahwa sistem rekomendasi berbasis konten yang diterapkan cukup efektif dalam menghasilkan rekomendasi yang relevan dan sesuai dengan preferensi pengguna.

Hasil ini memberikan gambaran bahwa pengguna dapat memperoleh pengalaman berbelanja yang lebih personal dan memuaskan melalui sistem rekomendasi ini. Dengan mampu menampilkan produk-produk yang secara signifikan mirip dengan preferensi pengguna, peluang untuk meningkatkan kepuasan pelanggan serta konversi penjualan menjadi lebih besar.

Namun demikian, perlu diingat bahwa kesuksesan suatu sistem rekomendasi tidak hanya ditentukan oleh kemiripan produk, tetapi juga oleh berbagai faktor lainnya seperti kebutuhan dan preferensi pengguna yang dapat berubah dari waktu ke waktu. Oleh karena itu, perlu adanya upaya terus-menerus dalam pengembangan dan peningkatan sistem rekomendasi ini agar dapat tetap relevan dan efektif dalam menghadapi dinamika pasar dan kebutuhan pelanggan.

### Deploy Model

In [68]:
with open('C:/Users/ASUS/kode/Projek/Recommendation_System_Manettas/df_baru.pkl', 'wb') as file:
    pickle.dump(df_baru, file)
with open('C:/Users/ASUS/kode/Projek/Recommendation_System_Manettas/count_vectorizer_model.pkl', 'wb') as file:
    pickle.dump(cv, file)
with open('C:/Users/ASUS/kode/Projek/Recommendation_System_Manettas/cosine_similarity_model.pkl', 'wb') as file:
    pickle.dump(kesamaan, file)
with open('C:/Users/ASUS/kode/Projek/Recommendation_System_Manettas/vektor_model.pkl', 'wb') as file:
    pickle.dump(vektor, file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>